In [ ]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
from matplotlib import colors
from time import sleep
from scipy.special import softmax
from mpl_toolkits.mplot3d import Axes3D

rg = np.random.RandomState(42)


In [ ]:

def row_col_to_seq(row_col, num_cols):  #Converts row_column to state number format
    return row_col[:,0] * num_cols + row_col[:,1]

def seq_to_col_row(seq, num_cols): #Converts state to row column format
    r = floor(seq / num_cols)
    c = seq - r * num_cols
    return np.array([[r, c]])

class GridWorld:
    """
    Creates a gridworld object to pass to an RL algorithm.
    Parameters
    ----------
    num_rows : int
        The number of rows in the gridworld.
    num_cols : int
        The number of cols in the gridworld.
    start_state : numpy array of shape (1, 2), np.array([[row, col]])
        The start state of the gridworld (can only be one start state)
    goal_states : numpy arrany of shape (n, 2)
        The goal states for the gridworld where n is the number of goal
        states.
    """
    def __init__(self, num_rows, num_cols, start_state, goal_states, wind = False):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.start_state = start_state
        self.goal_states = goal_states
        self.obs_states = None
        self.bad_states = None
        self.num_bad_states = 0
        self.p_good_trans = None
        self.bias = None
        self.r_step = None
        self.r_goal = None
        self.r_dead = None
        self.gamma = 1 # default is no discounting
        self.wind = wind
        self.done=False
        self.steps=0
        self.max_steps=100

    def add_obstructions(self, obstructed_states=None, bad_states=None, restart_states=None):

        self.obs_states = obstructed_states
        self.bad_states = bad_states
        if bad_states is not None:
            self.num_bad_states = bad_states.shape[0]
        else:
            self.num_bad_states = 0
        self.restart_states = restart_states
        if restart_states is not None:
            self.num_restart_states = restart_states.shape[0]
        else:
            self.num_restart_states = 0

    def add_transition_probability(self, p_good_transition, bias):

        self.p_good_trans = p_good_transition
        self.bias = bias

    def add_rewards(self, step_reward, goal_reward, bad_state_reward=None, restart_state_reward = None):

        self.r_step = step_reward
        self.r_goal = goal_reward
        self.r_bad = bad_state_reward
        self.r_restart = restart_state_reward


    def create_gridworld(self):

        self.num_actions = 4
        self.num_states = self.num_cols * self.num_rows# +1
        self.start_state_seq = row_col_to_seq(self.start_state, self.num_cols)
        self.goal_states_seq = row_col_to_seq(self.goal_states, self.num_cols)

        # rewards structure
        self.R = self.r_step * np.ones((self.num_states, 1))
        #self.R[self.num_states-1] = 0
        self.R[self.goal_states_seq] = self.r_goal

        for i in range(self.num_bad_states):
            if self.r_bad is None:
                raise Exception("Bad state specified but no reward is given")
            bad_state = row_col_to_seq(self.bad_states[i,:].reshape(1,-1), self.num_cols)
            #print("bad states", bad_state)
            self.R[bad_state, :] = self.r_bad
        for i in range(self.num_restart_states):
            if self.r_restart is None:
                raise Exception("Restart state specified but no reward is given")
            restart_state = row_col_to_seq(self.restart_states[i,:].reshape(1,-1), self.num_cols)
            #print("restart_state", restart_state)
            self.R[restart_state, :] = self.r_restart

        # probability model
        if self.p_good_trans == None:
            raise Exception("Must assign probability and bias terms via the add_transition_probability method.")

        self.P = np.zeros((self.num_states,self.num_states,self.num_actions))
        for action in range(self.num_actions):
            for state in range(self.num_states):


                # check if the state is the goal state or an obstructed state - transition to end
                row_col = seq_to_col_row(state, self.num_cols)
                if self.obs_states is not None:
                    end_states = np.vstack((self.obs_states, self.goal_states))
                else:
                    end_states = self.goal_states

                if any(np.sum(np.abs(end_states-row_col), 1) == 0):
                    self.P[state, state, action] = 1

                # else consider stochastic effects of action
                else:
                    for dir in range(-1,2,1):

                        direction = self._get_direction(action, dir)
                        next_state = self._get_state(state, direction)
                        if dir == 0:
                            prob = self.p_good_trans
                        elif dir == -1:
                            prob = (1 - self.p_good_trans)*(self.bias)
                        elif dir == 1:
                            prob = (1 - self.p_good_trans)*(1-self.bias)

                        self.P[state, next_state, action] += prob

                # make restart states transition back to the start state with
                # probability 1
                if self.restart_states is not None:
                    if any(np.sum(np.abs(self.restart_states-row_col),1)==0):
                        next_state = row_col_to_seq(self.start_state, self.num_cols)
                        self.P[state,:,:] = 0
                        self.P[state,next_state,:] = 1
        return self

    def render(self,state, render_agent=False, ax=None):
        grid = np.zeros((self.num_rows, self.num_cols), dtype=int)

        for goal in self.goal_states:
            grid[goal[0], goal[1]] = 4

        if self.bad_states is not None:
            for bad_state in self.bad_states:
                grid[bad_state[0], bad_state[1]] = 3

        if self.obs_states is not None:
            for obs_state in self.obs_states:
                grid[obs_state[0], obs_state[1]] = 2

        if self.restart_states is not None:
            for restart_state in self.restart_states:

                grid[restart_state[0], restart_state[1]]=6

        if render_agent:
            grid[state[0][0],state[0][1]] = 5

        plt.clf()

        if not render_agent:
            cmap = plt.cm.colors.ListedColormap(['#F5E5E1', '#F2A494', '#FF2D00', '#0004FF', '#00FF23'])
        else:
            cmap = plt.cm.colors.ListedColormap(['#F5E5E1', '#F2A494', '#FF2D00', '#0004FF', '#00FF23', '#F0FF00','#808080'])

        if ax is None:
            fig, ax = plt.subplots()

        ax.pcolor(grid, cmap=cmap, edgecolors='k', linewidths=2)

    def plot_Q(self, Q, message="Q plot"):
        plt.figure(figsize=(5, 5))
        plt.title(message)
        plt.pcolor(Q.max(-1), edgecolors='k', linewidths=2)
        plt.colorbar()

        def x_direct(a):
            if a in [0, 1]:
                return 0
            return 1 if a == 3 else -1

        def y_direct(a):
            if a in [3, 2]:
                return 0
            return -1 if a == 0 else 1

        policy = Q.argmax(-1)
        policyx = np.vectorize(x_direct)(policy)
        policyy = np.vectorize(y_direct)(policy)
        idx = np.indices(policy.shape)
        plt.quiver(idx[1].ravel() + 0.5, idx[0].ravel() + 0.5, policyx.ravel(), policyy.ravel(), pivot="middle",
                color='red')
        plt.show()




    def _get_direction(self, action, direction):

        left = [2,3,1,0]
        right = [3,2,0,1]
        if direction == 0:
            new_direction = action
        elif direction == -1:
            new_direction = left[action]
        elif direction == 1:
            new_direction = right[action]
        else:
            raise Exception("getDir received an unspecified case")
        return new_direction

    def _get_state(self, state, direction):

        row_change = [-1,1,0,0]
        col_change = [0,0,-1,1]
        row_col = seq_to_col_row(state, self.num_cols)
        row_col[0,0] += row_change[direction]
        row_col[0,1] += col_change[direction]

        # check for invalid states
        if self.obs_states is not None:
            if (np.any(row_col < 0) or
                np.any(row_col[:,0] > self.num_rows-1) or
                np.any(row_col[:,1] > self.num_cols-1) or
                np.any(np.sum(abs(self.obs_states - row_col), 1)==0)):
                next_state = state
            else:
                next_state = row_col_to_seq(row_col, self.num_cols)[0]
        else:
            if (np.any(row_col < 0) or
                np.any(row_col[:,0] > self.num_rows-1) or
                np.any(row_col[:,1] > self.num_cols-1)):
                next_state = state
            else:
                next_state = row_col_to_seq(row_col, self.num_cols)[0]

        return next_state
    


    def reset(self):
      self.done=False
      self.steps=0
      return int(self.start_state_seq)
      

    def step(self, state, action):
        p, r = 0, np.random.random()
        for next_state in range(self.num_states):

            p += self.P[state, next_state, action]

            if r <= p:
                break
        self.steps+=1

        if(self.wind and np.random.random() < 0.4):

          arr = self.P[next_state, :, 3]
          next_next = np.where(arr == np.amax(arr))
          next_next = next_next[0][0]
          if next_next in self.goal_states_seq or self.steps>=self.max_steps:
              self.done=True
          else:
              self.done=False
          
          return next_next, self.R[next_next]
          
        else:
          if next_state in self.goal_states_seq or self.steps>=self.max_steps:
              self.done=True
          else:
              self.done=False
          return next_state, self.R[next_state]


In [ ]:
def generate_world(windy,p_good_transition,start_state):
# specify world parameters
  wind=windy
  p_good_transition=p_good_transition
  start_state=np.array([start_state])

  num_cols = 10
  num_rows = 10
  obstructions = np.array([[0,7],[1,1],[1,2],[1,3],[1,7],[2,1],[2,3],
                          [2,7],[3,1],[3,3],[3,5],[4,3],[4,5],[4,7],
                          [5,3],[5,7],[5,9],[6,3],[6,9],[7,1],[7,6],
                          [7,7],[7,8],[7,9],[8,1],[8,5],[8,6],[9,1]])
  bad_states = np.array([[1,9],[4,2],[4,4],[7,5],[9,9]])
  restart_states = np.array([[3,7],[8,2]])
  #start_state = np.array([[0,4]])
  goal_states = np.array([[0,9],[2,2],[8,7]])

  # create model

  gw = GridWorld(num_rows=num_rows,
                  num_cols=num_cols,
                  start_state=start_state,
                  goal_states=goal_states, wind = wind)
  gw.add_obstructions(obstructed_states=obstructions,
                        bad_states=bad_states,
                        restart_states=restart_states)
  gw.add_rewards(step_reward=-1,
                  goal_reward=10,
                  bad_state_reward=-6,
                  restart_state_reward=-100)
  gw.add_transition_probability(p_good_transition=p_good_transition,
                                  bias=0.5)
  env = gw.create_gridworld()
  return env

def visualizing_env(env):
  print("Number of actions", env.num_actions) #0 -> UP, 1-> DOWN, 2 -> LEFT, 3-> RIGHT
  print("Number of states", env.num_states)
  print("start state", env.start_state_seq)
  print("goal state(s)", env.goal_states_seq)
  env.P[0,:,0]
  env.render(state=env.start_state,ax=plt,render_agent=True) # red = obstructions, green= goal states, blue= bad states, yellow= start state, grey= restart state,


# Defining the Action Selection Policies 



# Epsilon greedy
def choose_action_epsilon(env,Q, state, epsilon, rg=rg):
    actions=np.arange(env.num_actions)
    if not Q[state[0][0], state[0][1]].any() or rg.rand()<epsilon:
        return rg.choice(actions)
    else:
        return np.argmax(Q[state[0][0],state[0][1]])

# Softmax
def choose_action_softmax(env,Q, state, tau=1, rg=rg):
    actions = np.arange(env.num_actions)
    state_rowcol=seq_to_col_row(state, env.num_cols)
    q_values = Q[state_rowcol[0][0], state_rowcol[0][1]]
    
    # Apply softmax with temperature tau
    probabilities = softmax(q_values / tau)
    
    # Choose action based on probabilities
    chosen_action = rg.choice(actions, 1, p=probabilities)[0]
    
    return chosen_action


In [ ]:

# SARSA with Softmax

print_freq = 100

def sarsa_s(env, Q, gamma = 0.9, plot_heat = False, choose_action = choose_action_softmax,epsilon0=0.1,alpha0=0.3,tau=0.2):

    episode_rewards = np.zeros(episodes)
    steps_to_completion = np.zeros(episodes)
    if plot_heat:
        clear_output(wait=True)
        env.plot_Q(Q)
    epsilon = epsilon0
    alpha = alpha0
    #print(alpha,epsilon,gamma,tau)
    state_visit_counts = np.zeros((env.num_rows, env.num_cols))
    for ep in tqdm(range(episodes)):
        tot_reward, steps = 0, 0

        # Reset environment
        state = env.reset()
        action = choose_action(env,Q, state,tau)
        done = False
    
        while not env.done:
            state_next, reward = env.step(state,action)
            state_next_rowcol=seq_to_col_row(state_next, env.num_cols)
            state_rowcols=seq_to_col_row(state, env.num_cols)
            
            action_next = choose_action(env,Q, state_next,tau)


            Q[state_rowcols[0][0],state_rowcols[0][1], action] = Q[state_rowcols[0][0],state_rowcols[0][1], action] + alpha * (reward + gamma * Q[state_next_rowcol[0][0],state_next_rowcol[0][1], action_next] - Q[state_rowcols[0][0],state_rowcols[0][1], action])

            state_visit_counts[state_rowcols[0][0], state_rowcols[0][1]] += 1
       

            tot_reward += reward
            steps += 1

            state, action = state_next, action_next

        episode_rewards[ep] = tot_reward
        steps_to_completion[ep] = steps

        if (ep+1)%print_freq == 0 and plot_heat:
            clear_output(wait=True)
            env.plot_Q(Q, message = "Episode %d: Reward: %f, Steps: %.2f, Qmax: %.2f, Qmin: %.2f"%(ep+1, np.mean(episode_rewards[ep-print_freq+1:ep]),
                                                                           np.mean(steps_to_completion[ep-print_freq+1:ep]),
                                                                           Q.max(), Q.min()))

    return Q, episode_rewards, steps_to_completion,state_visit_counts

def best_policy(env,Q):

    state = env.reset()
    done = False
    steps = 0
    tot_reward = 0
    while not env.done:
        clear_output(wait=True)
        plt.figure(figsize=(10, 10))
        state_row_cols= seq_to_col_row(state, env.num_cols)
        env.render(state_row_cols,ax=plt, render_agent=True)
        plt.show()
        steps += 1
        state, reward = env.step(state,Q[state_row_cols[0][0],state_row_cols[0][1]].argmax())
        tot_reward += reward
        sleep(0.2)
    print("Steps: %d, Total Reward: %d"%(steps, tot_reward))


def qlearning_s(env, Q, gamma = 1, plot_heat = False, choose_action = choose_action_softmax,epsilon0=0.1,alpha0=0.3,tau=0.2):
  
    episode_rewards = np.zeros(episodes)
    steps_to_completion = np.zeros(episodes)
    state_visit_counts = np.zeros((env.num_rows, env.num_cols))
    if plot_heat:
        clear_output(wait=True)
        env.plot_Q(Q)
    epsilon = epsilon0
    alpha = alpha0
    for ep in tqdm(range(episodes)):
        tot_reward, steps = 0, 0

        # Reset environment
        state = env.reset()
        action = choose_action(env,Q, state,tau)
        done = False
    
        while not env.done:
            state_next, reward = env.step(state,action)
            state_next_rowcol=seq_to_col_row(state_next, env.num_cols)
            state_rowcols=seq_to_col_row(state, env.num_cols)
            
            action_next = choose_action(env,Q, state_next,tau)


            Q[state_rowcols[0][0],state_rowcols[0][1], action] = Q[state_rowcols[0][0],state_rowcols[0][1], action] + alpha * (reward + gamma * Q[state_next_rowcol[0][0],state_next_rowcol[0][1],np.argmax(Q[state_next_rowcol[0][0],state_next_rowcol[0][1]])] - Q[state_rowcols[0][0],state_rowcols[0][1], action])

            tot_reward += reward
            steps += 1
            state_visit_counts[state_rowcols[0][0], state_rowcols[0][1]] += 1

            state, action = state_next, action_next

        episode_rewards[ep] = tot_reward
        steps_to_completion[ep] = steps

        if (ep+1)%print_freq == 0 and plot_heat:
            clear_output(wait=True)
            env.plot_Q(Q, message = "Episode %d: Reward: %f, Steps: %.2f, Qmax: %.2f, Qmin: %.2f"%(ep+1, np.mean(episode_rewards[ep-print_freq+1:ep]),
                                                                           np.mean(steps_to_completion[ep-print_freq+1:ep]),
                                                                           Q.max(), Q.min()))

    return Q, episode_rewards, steps_to_completion,state_visit_counts






In [ ]:
# sarsa with epsilon greedy 

def sarsa_e(env, Q, gamma = 0.9, plot_heat = False, choose_action = choose_action_epsilon,epsilon0=0.1,alpha0=0.3,tau=0.2):

    episode_rewards = np.zeros(episodes)
    steps_to_completion = np.zeros(episodes)
    if plot_heat:
        clear_output(wait=True)
        env.plot_Q(Q)
    epsilon = epsilon0
    alpha = alpha0
    state_visit_counts = np.zeros((env.num_rows, env.num_cols))
    for ep in tqdm(range(episodes)):
        tot_reward, steps = 0, 0

        # Reset environment
        state = env.reset()
        action = choose_action(env,Q, state,epsilon)
        done = False
    
        while not env.done:
            state_next, reward = env.step(state,action)
            state_next_rowcol=seq_to_col_row(state_next, env.num_cols)
            state_rowcols=seq_to_col_row(state, env.num_cols)
            
            action_next = choose_action(env,Q, state_next,epsilon)


            Q[state_rowcols[0][0],state_rowcols[0][1], action] = Q[state_rowcols[0][0],state_rowcols[0][1], action] + alpha * (reward + gamma * Q[state_next_rowcol[0][0],state_next_rowcol[0][1], action_next] - Q[state_rowcols[0][0],state_rowcols[0][1], action])

            state_visit_counts[state_rowcols[0][0], state_rowcols[0][1]] += 1
       

            tot_reward += reward
            steps += 1

            state, action = state_next, action_next

        episode_rewards[ep] = tot_reward
        steps_to_completion[ep] = steps

        if (ep+1)%print_freq == 0 and plot_heat:
            clear_output(wait=True)
            env.plot_Q(Q, message = "Episode %d: Reward: %f, Steps: %.2f, Qmax: %.2f, Qmin: %.2f"%(ep+1, np.mean(episode_rewards[ep-print_freq+1:ep]),
                                                                           np.mean(steps_to_completion[ep-print_freq+1:ep]),
                                                                           Q.max(), Q.min()))

    return Q, episode_rewards, steps_to_completion,state_visit_counts
    



# Q-Learning with epsilon-greedy

def qlearning_e(env, Q, gamma = 0.9, plot_heat = False, choose_action = choose_action_epsilon,epsilon0=0.1,alpha0=0.3,tau=0.2):

    episode_rewards = np.zeros(episodes)
    steps_to_completion = np.zeros(episodes)
    state_visit_counts = np.zeros((env.num_rows, env.num_cols))
    if plot_heat:
        clear_output(wait=True)
        env.plot_Q(Q)
    epsilon = epsilon0
    alpha = alpha0
    for ep in tqdm(range(episodes)):
        tot_reward, steps = 0, 0

        # Reset environment
        state = env.reset()
        action = choose_action(env,Q, state,epsilon)
        done = False
    
        while not env.done:
            state_next, reward = env.step(state,action)
            state_next_rowcol=seq_to_col_row(state_next, env.num_cols)
            state_rowcols=seq_to_col_row(state, env.num_cols)
            
            action_next = choose_action(env,Q, state_next,epsilon)


            Q[state_rowcols[0][0],state_rowcols[0][1], action] = Q[state_rowcols[0][0],state_rowcols[0][1], action] + alpha * (reward + gamma * Q[state_next_rowcol[0][0],state_next_rowcol[0][1],np.argmax(Q[state_next_rowcol[0][0],state_next_rowcol[0][1]])] - Q[state_rowcols[0][0],state_rowcols[0][1], action])

            tot_reward += reward
            steps += 1
            state_visit_counts[state_rowcols[0][0], state_rowcols[0][1]] += 1

            state, action = state_next, action_next

        episode_rewards[ep] = tot_reward
        steps_to_completion[ep] = steps

        if (ep+1)%print_freq == 0 and plot_heat:
            clear_output(wait=True)
            env.plot_Q(Q, message = "Episode %d: Reward: %f, Steps: %.2f, Qmax: %.2f, Qmin: %.2f"%(ep+1, np.mean(episode_rewards[ep-print_freq+1:ep]),
                                                                           np.mean(steps_to_completion[ep-print_freq+1:ep]),
                                                                           Q.max(), Q.min()))
            
    return Q, episode_rewards, steps_to_completion,state_visit_counts



In [ ]:
def plot_all(func,env):
    env=env
    num_expts = 5
    reward_avgs, steps_avgs = [], []
    rewards_all,steps_all=[],[]
    mean_rewd=[]
    Q_all= np.zeros((env.num_rows, env.num_cols, env.num_actions))
    state_visits_all= np.zeros((env.num_rows, env.num_cols))
    for i in range(num_expts):
        print("Experiment: %d"%(i+1))
        Q = np.zeros((env.num_rows, env.num_cols, env.num_actions))
        rg = np.random.RandomState(i)
        Q, rewards, steps,state_visits = func(env, Q, gamma = gamma, plot_heat=False, choose_action= choose_action_softmax,epsilon0=epsilon0,alpha0=alpha0,tau=tau)
        rewards_all.append(rewards)
        mean_rewd.append(np.mean(rewards))
        steps_all.append(steps)
        Q_all=Q_all+Q
        state_visits_all+=state_visits

    Q_avg=Q_all/num_expts
    state_visits_avg=state_visits_all/num_expts
    reward_avgs.append(np.mean(rewards_all,axis=0))
    steps_avgs.append(np.mean(steps_all,axis=0))
    mean_steps=np.mean(steps_all)
    mean_rewards=np.mean(mean_rewd)

    env.plot_Q(Q_avg)

    episodes_no=np.arange(episodes)
    reward_stds = np.std(rewards_all, axis=0)
    steps_stds = np.std(steps_all, axis=0)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot Mean Steps
    axes[0].errorbar(episodes_no, steps_avgs[0], yerr=steps_stds, label='Mean Steps', fmt='-o', ecolor='skyblue')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Number of steps to Goal')
    axes[0].set_title('Mean Steps')

    # Plot Mean Reward
    axes[1].errorbar(episodes_no, reward_avgs[0], yerr=reward_stds, label='Mean Reward', fmt='-o', ecolor='skyblue')
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Total Reward')
    axes[1].set_title('Mean Reward')

    # Plot State Visit Counts Heatmap
    heatmap = axes[2].imshow(state_visits_avg, cmap='Blues', interpolation='nearest', origin='lower')
    axes[2].set_title('State Visit Counts Heatmap')
    axes[2].set_xlabel('Column Index')
    axes[2].set_ylabel('Row Index')
    axes[2].grid(True, linestyle='--', alpha=0.5, color='black') 
    fig.colorbar(heatmap, ax=axes[2], label='Visit Counts')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()

    # Show the plots
    plt.show()

    print('Average reward across all the runs = ',mean_rewards)
    print('Average steps across all the runs = ',mean_steps)

In [ ]:
def opt_all(func,env,individual,e_s):
    env=env
    num_expts = 1
    epsilon = 0
    tau = 0
    if e_s == 0:
        alpha, gamma, epsilon = individual['alpha'], individual['gamma'], individual['epsilon_tau']
    else:
        alpha, gamma, tau = individual['alpha'], individual['gamma'], individual['epsilon_tau']
    reward_avgs, steps_avgs = [], []
    rewards_all,steps_all=[],[]
    mean_rewd=[]
    #print(alpha,gamma,epsilon,tau)
    Q_all= np.zeros((env.num_rows, env.num_cols, env.num_actions))
    state_visits_all= np.zeros((env.num_rows, env.num_cols))
    for i in range(num_expts):
        print("Experiment: %d"%(i+1))
        Q = np.zeros((env.num_rows, env.num_cols, env.num_actions))
        rg = np.random.RandomState(i)
        Q, rewards, steps,state_visits = func(env, Q, gamma = gamma, plot_heat=False, choose_action= choose_action_softmax,epsilon0=epsilon,alpha0=alpha,tau=tau)
        rewards_all.append(rewards)
        mean_rewd.append(np.mean(rewards))
        steps_all.append(steps)
        Q_all=Q_all+Q
        state_visits_all+=state_visits

    Q_avg=Q_all/num_expts
    state_visits_avg=state_visits_all/num_expts
    reward_avgs.append(np.mean(rewards_all,axis=0))
    steps_avgs.append(np.mean(steps_all,axis=0))
    mean_steps=np.mean(steps_all)
    print(mean_steps)
    mean_rewards=np.mean(mean_rewd)
    return mean_rewards,mean_steps

In [ ]:
env1= generate_world(False,1,[0,4])
env2= generate_world(False,1,[3,6])
env3= generate_world(False,0.7,[0,4])
env4= generate_world(False,0.7,[3,6])
env5= generate_world(True,1,[0,4])
env6= generate_world(True,1,[3,6])

In [ ]:

def initialize_population(population_size,e_s):
    population = []
    for _ in range(population_size):
        if e_s:
            individual = {
                'alpha': np.random.uniform(0.1, 1.0),
                'gamma': np.random.uniform(0.8, 1.0),
                'epsilon_tau': np.random.uniform(0.3, 1), # tau for e_s=1
                # 'tau': np.random.uniform(0.5,0.8)
            }
            population.append(individual)
        else:
            individual = {
                'alpha': np.random.uniform(0.1, 0.7),
                'gamma': np.random.uniform(0.65, 1.0),
                'epsilon_tau': np.random.uniform(0.05,0.3)
            }
            population.append(individual)
    return population

def evaluate_individual(individual, env, func,e_s):
    alpha, gamma, epsilon_tau = individual['alpha'], individual['gamma'], individual['epsilon_tau']
    #print(alpha,gamma, epsilon, tau)
    mean_rewards_ok, mean_steps_ok = opt_all(func, env,individual,e_s)
    return mean_rewards_ok, mean_steps_ok

def combined_score(individual, rewards_weight=0.5, steps_weight=0.5):
    return rewards_weight * individual['mean_rewards'] + steps_weight * (-individual['mean_steps'] + 10)



def select_parents(population, num_parents):
    # Select top individuals based on their fitness
    # sorted_population = sorted(population, key=lambda x: (x['mean_rewards'], -x['mean_steps']), reverse=True)
    sorted_population = sorted(population, key=lambda x: combined_score(x), reverse=True)
    print(sorted_population[:num_parents])
    return sorted_population[:num_parents]

def crossover(parent1, parent2):
    # Perform crossover to create a new individual
    crossover_point = np.random.choice(list(parent1.keys()))
    child = {key: parent1[key] if np.random.rand() < 0.5 else parent2[key] for key in parent1.keys()}
    return child

def mutate(individual, mutation_rate=0.2,e_s=0):
    # Perform mutation on an individual
    for key in individual.keys():
        if np.random.rand() < mutation_rate:
            if key == 'epsilon_tau' and e_s == 1:
                individual[key] = np.random.uniform(0.5,1.0)
            elif key == 'alpha':
                individual[key] = np.random.uniform(0.6,1.0)
            elif key=='gamma':
                individual[key] = np.random.uniform(0.7,1.0)

            else:
                individual[key] = np.random.uniform(0.001, 0.25)
    return individual

def genetic_algorithm(env, func, population_size=15, num_generations=7, num_parents=5,e_s = 0):
    # Initialize population
    #if e_s = 1, then it is softmax, e_s = 0 then it is epsilon greedy
    population = initialize_population(population_size,e_s)
    all_parents = []
    # Evolution loop
    for generation in range(num_generations):
        print(f"\nGeneration {generation + 1}")
        for individual in population:
            # Evaluate fitness of each individual
            mean_rewards_yes, mean_steps_yes = evaluate_individual(individual, env, func,e_s)
            individual['mean_rewards'] = mean_rewards_yes

            individual['mean_steps'] = mean_steps_yes

        # Select parents
        parents = select_parents(population, num_parents)
        all_parents += parents
        best_individual = parents[0]
        best_hyperparameters = {key: best_individual[key] for key in ['alpha', 'gamma', 'epsilon_tau']}
        best_mean_rewards, best_mean_steps = best_individual['mean_rewards'], best_individual['mean_steps']
      
        print(best_individual)
        
        print(f"\nBest Hyperparameters: {best_hyperparameters}")
        print(f"Best Mean Rewards: {best_mean_rewards}")
        print(f"Best Mean Steps: {best_mean_steps}")

        # Create next generation
        new_population = parents.copy()
        while len(new_population) < population_size:
            parent1, parent2 = np.random.choice(parents, size=2, replace=False)
            child = crossover(parent1, parent2)
            child = mutate(child,e_s=e_s)
            new_population.append(child)

        # Update population for the next generation
        population = new_population
    

    return best_hyperparameters, best_mean_rewards, best_mean_steps, all_parents


In [ ]:
def plot_training(all_parents):

  x = np.array([individual['alpha'] for individual in all_parents])
  y = np.array([individual['gamma'] for individual in all_parents])
  z = np.array([individual['mean_rewards'] for individual in all_parents])
  w = np.array([individual['mean_steps'] for individual in all_parents])

  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(221)
  ax.set_xlabel('Alpha')
  ax.set_ylabel('mean_rewards')
  ax.scatter(x, z)  # You can customize the line style here if needed
  ax2 = fig.add_subplot(222)
  ax2.scatter(x,w)
  ax2.set_xlabel('Alpha')
  ax2.set_ylabel('mean_steps')
  ax3 = fig.add_subplot(223)
  ax3.set_xlabel('gamma')
  ax3.set_ylabel('mean_rewards')
  ax3.scatter(y, z, marker='o')  
  ax4 = fig.add_subplot(224)
  ax4.scatter(y,w,marker='o')
  ax4.set_xlabel('gamma')
  ax4.set_ylabel('mean_rewards')
  plt.show()

def write_results_to_file(file_path, algorithm_results, algorithm_name):
    with open(file_path, 'a') as file:
        file.write(f"{algorithm_name} Results:\n")
        file.write("Best Hyperparameters:\n")
        file.write(str(algorithm_results[0]) + '\n')
        file.write(f"Best Mean Rewards: {algorithm_results[1]}\n")
        file.write(f"Best Mean Steps: {algorithm_results[2]}\n\n")

file_path = "results1.txt"

In [ ]:
episodes=5000

In [ ]:
best_hyperparameters, best_mean_rewards, best_mean_steps, b1 = genetic_algorithm(env6, sarsa_s, e_s=1)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "SARSA_Softmax_e6")


In [ ]:

best_hyperparameters, best_mean_rewards, best_mean_steps, b2 = genetic_algorithm(env6, qlearning_s, e_s=1)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "Q-Learning_Softmax_e6")


In [ ]:

best_hyperparameters, best_mean_rewards, best_mean_steps, b3 = genetic_algorithm(env6, sarsa_e, e_s=0)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "SARSA_EpsilonGreedy_e6")


In [ ]:

best_hyperparameters, best_mean_rewards, best_mean_steps, b4 = genetic_algorithm(env6, qlearning_e, e_s=0)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "Q-Learning_EpsilonGreedy_e6")

In [44]:
best_hyperparameters, best_mean_rewards, best_mean_steps, a1 = genetic_algorithm(env1, sarsa_s, e_s=1)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "SARSA_Softmax_e1")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:14<00:00, 334.01it/s]


23.8406
Experiment: 1


100%|██████████| 5000/5000 [00:27<00:00, 181.40it/s]


34.5438
Experiment: 1


...

17.3836
Experiment: 1


100%|██████████| 5000/5000 [00:28<00:00, 172.92it/s]


37.2672
Experiment: 1


100%|██████████| 5000/5000 [00:09<00:00, 521.11it/s]

13.2938
[{'alpha': 0.6603787507134097, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535, 'mean_rewards': -6.671, 'mean_steps': 12.5512}, {'alpha': 0.21118892678947387, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535, 'mean_rewards': -7.2496, 'mean_steps': 13.107}, {'alpha': 0.21118892678947387, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535, 'mean_rewards': -7.389, 'mean_steps': 13.2246}, {'alpha': 0.21118892678947387, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535, 'mean_rewards': -7.4086, 'mean_steps': 13.2698}, {'alpha': 0.21118892678947387, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535, 'mean_rewards': -7.4438, 'mean_steps': 13.2788}]
{'alpha': 0.6603787507134097, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535, 'mean_rewards': -6.671, 'mean_steps': 12.5512}

Best Hyperparameters: {'alpha': 0.6603787507134097, 'gamma': 0.9864573742737344, 'epsilon_tau': 0.46052088325714535}
Best Mean 

In [45]:
best_hyperparameters, best_mean_rewards, best_mean_steps, a2 = genetic_algorithm(env1, qlearning_s, e_s=1)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "Q-Learning_Softmax_e1")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:25<00:00, 192.68it/s]


39.0184
Experiment: 1


100%|██████████| 5000/5000 [00:10<00:00, 463.04it/s]


17.4922
Experiment: 1


...

23.0064
Experiment: 1


100%|██████████| 5000/5000 [00:10<00:00, 460.77it/s]


17.433
Experiment: 1


100%|██████████| 5000/5000 [00:11<00:00, 452.54it/s]

17.7192
[{'alpha': 0.7679204513529706, 'gamma': 0.9640846284392658, 'epsilon_tau': 0.3737313381369028, 'mean_rewards': -6.657, 'mean_steps': 12.5446}, {'alpha': 0.7755101743709872, 'gamma': 0.987529346538963, 'epsilon_tau': 0.3737313381369028, 'mean_rewards': -6.54, 'mean_steps': 17.3912}, {'alpha': 0.7755101743709872, 'gamma': 0.987529346538963, 'epsilon_tau': 0.3737313381369028, 'mean_rewards': -6.6046, 'mean_steps': 17.433}, {'alpha': 0.7679204513529706, 'gamma': 0.9640846284392658, 'epsilon_tau': 0.3737313381369028, 'mean_rewards': -6.6036, 'mean_steps': 17.442}, {'alpha': 0.7679204513529706, 'gamma': 0.9640846284392658, 'epsilon_tau': 0.3737313381369028, 'mean_rewards': -6.6218, 'mean_steps': 17.4486}]
{'alpha': 0.7679204513529706, 'gamma': 0.9640846284392658, 'epsilon_tau': 0.3737313381369028, 'mean_rewards': -6.657, 'mean_steps': 12.5446}

Best Hyperparameters: {'alpha': 0.7679204513529706, 'gamma': 0.9640846284392658, 'epsilon_tau': 0.3737313381369028}
Best Mean Rewards: -6.657

In [46]:
best_hyperparameters, best_mean_rewards, best_mean_steps, a3 = genetic_algorithm(env1, sarsa_e, e_s=0)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "SARSA_EpsilonGreedy_e1")



Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:19<00:00, 259.40it/s]


36.8448
Experiment: 1


100%|██████████| 5000/5000 [00:17<00:00, 289.01it/s]


36.6548
Experiment: 1


...

17.3846
Experiment: 1


100%|██████████| 5000/5000 [00:08<00:00, 577.93it/s]


17.4046
Experiment: 1


100%|██████████| 5000/5000 [00:08<00:00, 616.39it/s]

17.354
[{'alpha': 0.7591830750919508, 'gamma': 0.9885114496433483, 'epsilon_tau': 0.01725410788350103, 'mean_rewards': -6.5112, 'mean_steps': 17.3514}, {'alpha': 0.7591830750919508, 'gamma': 0.9885114496433483, 'epsilon_tau': 0.01725410788350103, 'mean_rewards': -6.515, 'mean_steps': 17.3558}, {'alpha': 0.8282341354202208, 'gamma': 0.946529602064741, 'epsilon_tau': 0.01725410788350103, 'mean_rewards': -6.5164, 'mean_steps': 17.3754}, {'alpha': 0.8196490070391125, 'gamma': 0.946529602064741, 'epsilon_tau': 0.09735061865962202, 'mean_rewards': -6.5348, 'mean_steps': 17.3846}, {'alpha': 0.8282341354202208, 'gamma': 0.946529602064741, 'epsilon_tau': 0.21399628846551672, 'mean_rewards': -6.5358, 'mean_steps': 17.384}]
{'alpha': 0.7591830750919508, 'gamma': 0.9885114496433483, 'epsilon_tau': 0.01725410788350103, 'mean_rewards': -6.5112, 'mean_steps': 17.3514}

Best Hyperparameters: {'alpha': 0.7591830750919508, 'gamma': 0.9885114496433483, 'epsilon_tau': 0.01725410788350103}
Best Mean Reward

In [47]:
best_hyperparameters, best_mean_rewards, best_mean_steps, a4 = genetic_algorithm(env1, qlearning_e, e_s=0)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "Q-Learning_EpsilonGreedy_e1")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:11<00:00, 423.12it/s]


21.956
Experiment: 1


100%|██████████| 5000/5000 [00:29<00:00, 169.38it/s]


64.6992
Experiment: 1


...

17.3958
Experiment: 1


100%|██████████| 5000/5000 [00:09<00:00, 540.15it/s]


17.3362
Experiment: 1


100%|██████████| 5000/5000 [00:09<00:00, 547.17it/s]

17.4344
[{'alpha': 0.8113675182488771, 'gamma': 0.9887996327500127, 'epsilon_tau': 0.09284647480243868, 'mean_rewards': -6.5252, 'mean_steps': 12.4022}, {'alpha': 0.9366280070739533, 'gamma': 0.9960852362843509, 'epsilon_tau': 0.1715253133518598, 'mean_rewards': -6.4538, 'mean_steps': 17.3078}, {'alpha': 0.9366280070739533, 'gamma': 0.9887996327500127, 'epsilon_tau': 0.2148343739440326, 'mean_rewards': -6.4798, 'mean_steps': 17.3378}, {'alpha': 0.9366280070739533, 'gamma': 0.9887996327500127, 'epsilon_tau': 0.2148343739440326, 'mean_rewards': -6.4824, 'mean_steps': 17.3362}, {'alpha': 0.9366280070739533, 'gamma': 0.9960852362843509, 'epsilon_tau': 0.09284647480243868, 'mean_rewards': -6.4936, 'mean_steps': 17.3306}]
{'alpha': 0.8113675182488771, 'gamma': 0.9887996327500127, 'epsilon_tau': 0.09284647480243868, 'mean_rewards': -6.5252, 'mean_steps': 12.4022}

Best Hyperparameters: {'alpha': 0.8113675182488771, 'gamma': 0.9887996327500127, 'epsilon_tau': 0.09284647480243868}
Best Mean Rew

In [48]:
best_hyperparameters, best_mean_rewards, best_mean_steps, d1 = genetic_algorithm(env4, sarsa_s, e_s=1)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "SARSA_Softmax_e4")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:14<00:00, 354.16it/s]


27.7352
Experiment: 1


100%|██████████| 5000/5000 [00:36<00:00, 138.59it/s]


78.0096
Experiment: 1


...

26.1368
Experiment: 1


100%|██████████| 5000/5000 [00:13<00:00, 364.54it/s]


29.0286
Experiment: 1


100%|██████████| 5000/5000 [00:40<00:00, 124.12it/s]

88.4254
[{'alpha': 0.6653632729173063, 'gamma': 0.9908767051431533, 'epsilon_tau': 0.7133989663909417, 'mean_rewards': -20.9498, 'mean_steps': 26.1368}, {'alpha': 0.6200478383451511, 'gamma': 0.9885509604565297, 'epsilon_tau': 0.652518375803463, 'mean_rewards': -20.8394, 'mean_steps': 26.4146}, {'alpha': 0.6200478383451511, 'gamma': 0.9885509604565297, 'epsilon_tau': 0.9714030394107416, 'mean_rewards': -21.174, 'mean_steps': 26.2026}, {'alpha': 0.6200478383451511, 'gamma': 0.9885509604565297, 'epsilon_tau': 0.9714030394107416, 'mean_rewards': -21.2338, 'mean_steps': 26.2554}, {'alpha': 0.6200478383451511, 'gamma': 0.9885509604565297, 'epsilon_tau': 0.9714030394107416, 'mean_rewards': -20.99, 'mean_steps': 26.9368}]
{'alpha': 0.6653632729173063, 'gamma': 0.9908767051431533, 'epsilon_tau': 0.7133989663909417, 'mean_rewards': -20.9498, 'mean_steps': 26.1368}

Best Hyperparameters: {'alpha': 0.6653632729173063, 'gamma': 0.9908767051431533, 'epsilon_tau': 0.7133989663909417}
Best Mean Rewar

In [49]:
best_hyperparameters, best_mean_rewards, best_mean_steps, d2 = genetic_algorithm(env4, qlearning_s, e_s=1)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "Q-Learning_Softmax_e4")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:33<00:00, 147.99it/s]


68.9368
Experiment: 1


100%|██████████| 5000/5000 [00:23<00:00, 211.39it/s]


46.9008
Experiment: 1


...

30.0022
Experiment: 1


100%|██████████| 5000/5000 [00:11<00:00, 426.56it/s]


26.1738
Experiment: 1


100%|██████████| 5000/5000 [00:12<00:00, 408.78it/s]

27.252
[{'alpha': 0.739682318905825, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.5868202249558615, 'mean_rewards': -20.5504, 'mean_steps': 26.1738}, {'alpha': 0.609503583393099, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.5769556291969975, 'mean_rewards': -21.4092, 'mean_steps': 26.6228}, {'alpha': 0.609503583393099, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.5769556291969975, 'mean_rewards': -21.0358, 'mean_steps': 27.3148}, {'alpha': 0.609503583393099, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.35145021600216664, 'mean_rewards': -21.292, 'mean_steps': 27.252}, {'alpha': 0.609503583393099, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.5769556291969975, 'mean_rewards': -21.3814, 'mean_steps': 27.9304}]
{'alpha': 0.739682318905825, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.5868202249558615, 'mean_rewards': -20.5504, 'mean_steps': 26.1738}

Best Hyperparameters: {'alpha': 0.739682318905825, 'gamma': 0.9892010591527947, 'epsilon_tau': 0.5868202249558615}
Best Mean Rewards: -

In [50]:
best_hyperparameters, best_mean_rewards, best_mean_steps, d3 = genetic_algorithm(env4, sarsa_e, e_s=0)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "SARSA_EpsilonGreedy_e4")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:13<00:00, 362.33it/s]


31.8682
Experiment: 1


100%|██████████| 5000/5000 [00:31<00:00, 156.73it/s]


76.8484
Experiment: 1


...

31.6266
Experiment: 1


100%|██████████| 5000/5000 [00:12<00:00, 408.66it/s]


28.6314
Experiment: 1


100%|██████████| 5000/5000 [00:11<00:00, 419.95it/s]

28.46
[{'alpha': 0.35498033669497064, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.02116819161552588, 'mean_rewards': -19.8592, 'mean_steps': 25.9794}, {'alpha': 0.35498033669497064, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.07286054896877493, 'mean_rewards': -21.2138, 'mean_steps': 26.4156}, {'alpha': 0.35498033669497064, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.07286054896877493, 'mean_rewards': -20.932, 'mean_steps': 27.2092}, {'alpha': 0.35498033669497064, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.10606721485843337, 'mean_rewards': -20.9702, 'mean_steps': 27.4152}, {'alpha': 0.3733304297956709, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.18936318329203997, 'mean_rewards': -21.1022, 'mean_steps': 28.4786}]
{'alpha': 0.35498033669497064, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.02116819161552588, 'mean_rewards': -19.8592, 'mean_steps': 25.9794}

Best Hyperparameters: {'alpha': 0.35498033669497064, 'gamma': 0.9669457659729693, 'epsilon_tau': 0.02116819161552588}
B

In [51]:
best_hyperparameters, best_mean_rewards, best_mean_steps, d4 = genetic_algorithm(env4, qlearning_e, e_s=0)
write_results_to_file(file_path, (best_hyperparameters, best_mean_rewards, best_mean_steps), "Q-Learning_EpsilonGreedy_e4")


Generation 1
Experiment: 1


100%|██████████| 5000/5000 [00:12<00:00, 400.23it/s]


28.2522
Experiment: 1


100%|██████████| 5000/5000 [00:19<00:00, 250.03it/s]


46.218
Experiment: 1


...

28.1212
Experiment: 1


100%|██████████| 5000/5000 [00:12<00:00, 396.53it/s]


27.5864
Experiment: 1


100%|██████████| 5000/5000 [00:13<00:00, 357.29it/s]

30.157
[{'alpha': 0.5889695809039539, 'gamma': 0.9842975077898486, 'epsilon_tau': 0.11619440897560725, 'mean_rewards': -20.435, 'mean_steps': 25.8506}, {'alpha': 0.5889695809039539, 'gamma': 0.9842975077898486, 'epsilon_tau': 0.08412883577744186, 'mean_rewards': -20.264, 'mean_steps': 26.3394}, {'alpha': 0.23076706806487618, 'gamma': 0.976984590744896, 'epsilon_tau': 0.15667983520926923, 'mean_rewards': -20.1902, 'mean_steps': 26.6498}, {'alpha': 0.6389914990281362, 'gamma': 0.976984590744896, 'epsilon_tau': 0.08412883577744186, 'mean_rewards': -20.7836, 'mean_steps': 26.8212}, {'alpha': 0.5889695809039539, 'gamma': 0.9842975077898486, 'epsilon_tau': 0.11619440897560725, 'mean_rewards': -21.2614, 'mean_steps': 26.5898}]
{'alpha': 0.5889695809039539, 'gamma': 0.9842975077898486, 'epsilon_tau': 0.11619440897560725, 'mean_rewards': -20.435, 'mean_steps': 25.8506}

Best Hyperparameters: {'alpha': 0.5889695809039539, 'gamma': 0.9842975077898486, 'epsilon_tau': 0.11619440897560725}
Best Mean